<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/main/ex13noteA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex13noteA

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2022)


In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

----
## 次元削減(2) 主成分分析 - 中編
----




---
### 主成分分析の問題の解

主成分分析の目的は，線形変換によって，「変換後のデータの分散がなるべく大きくなるような（元のデータの分散を保つような）」次元削減を行うことです．
$D$次元のデータ $\mathbf{x}$ を1次元にする場合には，この変換は
$$
y = \mathbf{w}\cdot\mathbf{x}
$$
という形になります．前回の結論は，「ベクトル $\mathbf{w}$ をデータの分散共分散行列の最大固有値に対する固有ベクトルにすればよい」というものでした．
今回は，$D$次元のデータを $H$ 次元に削減するとして，$H \geq 2$ の場合にどうすべきかを考えます．


この場合，次元削減を行う線形変換は
$$
\mathbf{y} = W\mathbf{x}
$$
と表せます．前回説明したように，
$$
\mathbf{y} = \begin{pmatrix}
y_1\\
y_2\\
\vdots\\
y_H
\end{pmatrix} = \begin{pmatrix}
\mathbf{w}_1\cdot\mathbf{x}\\
\mathbf{w}_2\cdot\mathbf{x}\\
\vdots\\
\mathbf{w}_H\cdot\mathbf{x}
\end{pmatrix}
$$
となっていますので，このような線形変換 $W$ を定めることは，$H$個のベクトル $\mathbf{w}_1, \mathbf{w}_2,\ldots,\mathbf{w}_H$ を定めることと同じです．

このとき，$\mathbf{w}_1$は，上述の通り，データの分散共分散行列の最大固有値に対応する固有ベクトルとすることになります（下図左のピンクの矢線）．
ここから更に $\mathbf{w}_2$ 以降も定めるのですが，主成分分析では，$\mathbf{w}_1, \mathbf{w}_2,\ldots,\mathbf{w}_H$ は，互いに直交した向きにとります．そうしないと，$y_1, y_2, \ldots, y_H$ の間の相関がなくならず，変換後の分散を最大化することができないからです.
そのため，$\mathbf{w}_2$は，$\mathbf{w}_1$ と直交する向きの中から，その方向に抽出した特徴 $y_2 = \mathbf{w}_2\cdot\mathbf{x}$ の分散が最大になる向きとることになります．

下図中央の緑色の平面は，原点を通りかつピンクのベクトルと直交しています．$\mathbf{w}_2$ は，この平面内の様々な向きのベクトルの中から選ぶことになります．このとき，下図右のように，この緑の平面上でデータ点を眺めたときに分散が最も大きくなる方向を選べば，$y_2$の分散を最大化することができます．



<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/pcafig06.png">


以下，同様のことを繰り返せば $\mathbf{w}_3$ 以降も定められます．
すなわち，$\mathbf{w}_h$は，$\mathbf{w}_1,\ldots,\mathbf{w}_{h-1}$ と直交するという条件の下で，分散が最大になる向きにとります．
詳しい説明は省きますが，結局，主成分分析の問題の解は，次のようになります．



［主成分分析の問題の解］

$N$個の$D$次元ベクトルから成る学習データが与えられるとする．
$$
\{ \mathbf{x}_n \in {\cal R}^{D} | n = 1, 2, \ldots, N\}
$$
ただし，このデータの平均は $\mathbf{0}$ とする．
また，$\mathbf{x}$ の分散共分散行列 $V = \frac{1}{N}\sum_{n=1}^{N}\mathbf{x}_n\mathbf{x}_n^{\top}$ の固有値を $\lambda_1 \geq \lambda_2 \geq \ldots \geq \lambda_D$とし，これらに対応する単位固有ベクトルをそれぞれ $\mathbf{u}_1, \mathbf{u}_2,\ldots,\mathbf{u}_D$ とおく．

$H\times D$行列 $W$ によって
$$
\mathbf{y} = W\mathbf{x} = \begin{pmatrix}
\mathbf{w}_1\cdot\mathbf{x}\\
\mathbf{w}_2\cdot\mathbf{x}\\
\vdots\\
\mathbf{w}_H\cdot\mathbf{x}
\end{pmatrix}
$$
と次元削減するとき，主成分分析の目的を実現するためには，
ベクトル $\mathbf{w}_h$ ($h = 1, 2, \ldots, H$) を $\mathbf{u}_h$ の向きにとればよい．つまり，$\mathbf{w}_h = \mathbf{u}_h$ または $\mathbf{w}_h = -\mathbf{u}_h$ とすればよい．

---
### ［例題］数物情3次元データの主成分分析

「数学」「物理」「情報」の点数のデータに主成分分析を適用してみましょう．

In [ ]:
# 数物情データを入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/PIP/mpi100-mac.csv
dfMPI = pd.read_csv('mpi100-mac.csv', index_col=0)
datMPI = dfMPI.to_numpy().astype(float)
dfMPI

In [ ]:
# 平均
Xm = np.mean(dfMPI.to_numpy(), axis=0)
print('平均:', Xm)

# 平均を引いて NumPy 配列にする
X = dfMPI.to_numpy() - Xm
N, D = X.shape
print('X.shape:', X.shape) # X は 100 x 3
print(X[:5, :]) # 最初の5人分を表示

以下のセルを実行すると，`X` の分散共分散行列の固有値と固有ベクトルが求まります．
ここでは，特異値分解という手法を使ってそれらを求めていますが，説明は省きます．

In [ ]:
# 分散共分散行列の固有値と固有ベクトルを求める（Xの特異値分解経由で）
_, sval, Vt = np.linalg.svd(X, full_matrices=False)
eval = sval**2/N
U = Vt
for d in range(D):
    print(f'{d+1}番目の固有値:{eval[d]:.2f}   固有ベクトル:', U[d, :])

得られた3つの固有ベクトルのうち，1番目と2番目を使って行列 `W` を構成し，これを用いて3次元の `X` を 2次元の `Y` に変換します．

In [ ]:
W = U[:2, :]
print(W.shape)
print(W)
Y = X @ W.T # y = Wx の計算
print(Y.shape)
print(Y[:5, :]) # 最初の5人分を表示

`Y` の散布図を描いてみましょう．

In [ ]:
nList = [44, 47, 41, 6] # 特徴的な4人

fig, ax = plt.subplots(facecolor="white", figsize=(8, 8))
ax.scatter(Y[:, 0], Y[:, 1])
ax.scatter(Y[nList, 0], Y[nList, 1])
ax.axvline(0, linestyle='-', color='gray')
ax.axhline(0, linestyle='-', color='gray')
ax.set_xlim(-40, 40)
ax.set_ylim(-40, 40)
ax.set_aspect('equal')
ax.set_xlabel('$y_1$')
ax.set_ylabel('$y_2$')
ax.legend()
for n in nList:
    plt.annotate(f'{n}', (Y[n, 0]+2, Y[n, 1]+2))
plt.show()

図にオレンジ色の点として描かれている4人について，元の点数，Xの値，Y の値を表示すると次のようになります．

In [ ]:
nList = [44, 47, 41, 6]
for n in nList:
    print(f'{n:2d} {X[n]+Xm} {X[n]} {Y[n]}')

$y_1$ の値は，「数学が高得点で他の2科目が低得点な度合い」と解釈できます（ex12noteC も参照）．
44番や47番のひとは，他の科目より数学の点数の方がずっと高いようです．
逆に，41番や6番のひとは，数学よりは他の科目の方が高得点です．

$y_2$ の方は，
$$
y_2 = -0.3 (\mbox{数学}) - 0.9(\mbox{物理}) + 0.4(\mbox{情報})
$$
となっていることからわかるように，「情報が高得点で他の2科目が低得点な度合い」のようです．

ここで紹介した例は，元がたかだか3次元ですので，元のデータの散布図を描いたりすれば，次元削減を経ずに上記のような分析をすることもできなくはないかもしれません．しかし，もっと多次元のデータの場合，主成分分析によって次元削減したデータを分析することで，元のデータでは分かりづらかった性質を見出すことができたりします．